In [61]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

llm = ChatOpenAI(temperature=0.1, verbose=True)

# 데이터 로더 정의
loader = UnstructuredFileLoader("./files/example.txt")

# 텍스트 스플리터 정의


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", chunk_size=600, chunk_overlap=100
)

# 벡터 임베더 정의 + 캐싱 설정
embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=OpenAIEmbeddings(),
    document_embedding_cache=LocalFileStore("./.cache/"),
)

In [62]:
# 데이터 로딩하기
docs = loader.load()
# 텍스트 스플릿하기
split_docs = splitter.split_documents(docs)


# 임베딩해서 vecotr를 벡터스토어에 저장하고 + 캐싱하기
vectorstore = FAISS.from_documents(split_docs, embedder)
vectorstore.similarity_search("Where does the Winstion live?")

[Document(page_content='Part 1, Chapter 1\nPart One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varico

In [65]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


def show_prompt(inputs):
    print(inputs)
    return inputs


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_invoke(inputs):
    docs = inputs["docs"]
    question = inputs["question"]
    results = []
    results.append(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in docs
    )
    return "\n\n".join(results)


map_chain = {
    "docs": vectorstore.as_retriever(),
    "question": RunnablePassthrough(),
} | RunnableLambda(map_invoke)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {"context": map_chain, "question": RunnablePassthrough()}
    | final_prompt
    | RunnableLambda(show_prompt)
    | llm
)

chain.invoke("Describe Victory Mansions")

4
messages=[SystemMessage(content='\n            Given the following extracted parts of a long document and a question, create a final answer. \n            If you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\n            ------\n            Victory Mansions is not mentioned in the provided text.\n\nVictory Mansions is a building with glass doors that Winston Smith enters. The hallway has a smell of boiled cabbage and old rag mats. There is a large colored poster on one end of the hallway, depicting the face of a man in his forties with a black mustache and ruggedly handsome features. The flat is located on the seventh floor, and Winston, who is thirty-nine with a varicose ulcer, slowly makes his way up the stairs, resting several times. On each landing, there is a poster with the caption "BIG BROTHER IS WATCHING YOU." Inside the flat, there is a telescreen, an oblong metal plaque that functions as a dulled mirror on the right-hand wall. The v

AIMessage(content='The provided text does not provide a specific description of Victory Mansions.')